In [15]:
import requests
import json
import string
import getpass

username = 'bogdan.gherasim'
password = 'jk128pq39H'

base_url = 'http://www.fairdomhub.org/'
base_url_1 = 'https://testing.sysmo-db.org'


headers = {"Content-type": "application/vnd.api+json",
           "Accept": "application/vnd.api+json",
           "Accept-Charset": "ISO-8859-1"}

session = requests.Session()
session.headers.update(headers)
# session.auth = (input('Username:'), getpass.getpass('Password'))
session.auth = (username, password)


def remove_non_printable(text):
    return ''.join(i for i in text if i in string.printable)

def retrieveDataAPIRequest(ID, operation = 'assays'):
    print("Calling API for data on " + operation + ", requesting object with ID = " + ID)
    r = session.get(base_url + operation + '/' + ID)

    r.raise_for_status()
    
    
    attributes = r.json()['data']['attributes']
    relationships = r.json()['data']['relationships']
    
    title = attributes['title']
    
    description = attributes['description']
    
    classTitle = attributes['assay_class']['title']
    classDescription = attributes['assay_class']['description']
    
    typeLabel = attributes['assay_type']['label']
    typeURI = attributes['assay_type']['uri']
     
    technologyLabel = attributes['technology_type']['label']
    technologyURI = attributes['technology_type']['uri']

    
    creators = relationships['creators']['data']
    submitter = relationships['submitter']['data']
    organisms = relationships['organisms']['data']
    people = relationships['people']['data']
    projects = relationships['projects']['data']
    investigation = relationships['investigation']['data']
    study = relationships['study']['data']
    data_files = relationships['data_files']['data']
    models = relationships['models']['data']
    sops = relationships['sops']['data']
    publications = relationships['publications']['data']
    documents = relationships['documents']['data']
    
    
    result = {
        "title": title,
        "description": description,
        "classTitle": classTitle,
        "classDescription": classDescription,
        "typeLabel": typeLabel,
        "typeURI": typeURI,
        "technologyLabel": technologyLabel,
        "technologyURI": technologyURI,
        "creators": creators,
        "submitter": submitter,
        "organisms": organisms,
        "people": people,
        "projects": projects,
        "investigation": investigation,
        "study": study,
        "data_files": data_files,
        "models": models,
        "sops": sops,
        "publications": publications,
        "documents": documents
    }
    return result
def printDataAPIRequest():
    
    assay = retrieveDataAPIRequest("709")
    
    if (assay['classDescription'] == None): assay['classDescription'] = ""

    print("{0} ({2}) -- {4},{6}\n\n{1}\n\nLinks:{5}\n{7}".format(remove_non_printable(assay['title']),
                                               remove_non_printable(assay['description']),
                                               remove_non_printable(assay['classTitle']),
                                               remove_non_printable(assay['classDescription']),
                                               remove_non_printable(assay['typeLabel']),
                                               remove_non_printable(assay['typeURI']),
                                               remove_non_printable(assay['technologyLabel']),
                                               remove_non_printable(assay['technologyURI'])))
    print("\n\nCreators: ")
    for creator in assay['creators']:
        creatorData = retrieveDataAPIRequest(operation=creator['type'], ID=creator['id'])
        print("  {0}\n".format(remove_non_printable(creatorData['title'])))

## Authentication

## Searching

In [3]:
# 



term = input("Enter the search term: ")

payload = {'q' : term} # will give the parameter for the HTTP request as "?q=<<term>>"

r = requests.get(base_url + 'search', headers=headers, params=payload)

r.raise_for_status() # makes sure to throw err if bad request

for entry in r.json()['data'] :
    title = entry['attributes']['title']
    entry_type = entry['type']
    print ("Type: {0}   Title: {1}".format(remove_non_printable(entry_type),
                                            remove_non_printable(title)))

Enter the search term: jugu


ConnectionError: HTTPConnectionPool(host='www.fairdomhub.org', port=80): Max retries exceeded with url: /search?q=jugu (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fd7a0135470>: Failed to establish a new connection: [Errno -2] Name or service not known',))

## Reading info from people 

In [2]:
session = requests.Session()
session.headers.update(headers)
# session.auth = (input('Username:'), getpass.getpass('Password'))
session.auth = (username, password)

r = session.get(base_url + 'people/433')

r.raise_for_status()

attributes = r.json()['data']['attributes']
name = attributes['title']
position = attributes['project_positions'][0]['position_name']
description = attributes['description']
print ("{0} ( {1} ) \n\n{2}".format(remove_non_printable(name),
                                   remove_non_printable(position),
                                   remove_non_printable(description)))

ConnectionError: HTTPConnectionPool(host='www.fairdomhub.org', port=80): Max retries exceeded with url: /people/433 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fd6eda2acc0>: Failed to establish a new connection: [Errno -2] Name or service not known',))

In [ ]:
r.json()


## About assays

In [2]:
session = requests.Session()
session.headers.update(headers)
# session.auth = (input('Username:'), getpass.getpass('Password'))
session.auth = (username, password)

assayRequest = session.get(base_url + 'assays/709')

assayRequest.raise_for_status()

attributes = assayRequest.json()['data']['attributes']
relationships = assayRequest.json()['data']['relationships']
organisms = relationships['organisms']['data']
name = attributes['title']
assayType = attributes['assay_class']['title']
description = attributes['description']
assayType = attributes['assay_type']['label']
creators = relationships['creators']['data']

print ("{0} ( {1} ) - {3} \n\n{2}".format(remove_non_printable(name),
                                   remove_non_printable(assayType),
                                   remove_non_printable(description),
                                   remove_non_printable(assayType)))


print("\nContributors: ")
for person in creators:
    creatorsRequest = session.get(base_url + person['type'] + "/" + person['id'])
    creatorsRequest.raise_for_status()
    name = creatorsRequest.json()['data']['attributes']['title']
    print("{0},".format(remove_non_printable(name)))

    
print("\nOrganisms: ")
for organism in organisms:
    organismRequest = session.get(base_url + organism['type'] + "/" + organism['id'])
    organismRequest.raise_for_status()
    organismRequest.json()
    name = organismRequest.json()['data']['attributes']['title']
    print("Assay uses organism: {0}\nOther assays using this: \n".format(remove_non_printable(name)))
    
    assaysArray = organismRequest.json()['data']['relationships']['assays']['data']
    for assay in assaysArray:
#         print("{0}\n".format(assay['id']))
        relatedAssaysRequest = session.get(base_url + assay['type'] + "/" + assay['id'])
        relatedAssaysRequest.raise_for_status()
        name = relatedAssaysRequest.json()['data']['attributes']['title']
        print("{0}".format(remove_non_printable(name)))
    
    assaysArray = organismRequest.json()['data']['relationships']['assays']['data']
    for assay in assaysArray:
#         print("{0}\n".format(assay['id']))
        relatedAssaysRequest = session.get(base_url + assay['type'] + "/" + assay['id'])
        relatedAssaysRequest.raise_for_status()
        name = relatedAssaysRequest.json()['data']['attributes']['title']
        print("{0}".format(remove_non_printable(name)))

NameError: name 'remove_non_printable' is not defined

In [ ]:
r.json()

In [ ]:
organismRequest.json()

In [16]:
printDataAPIRequest()

Calling API for data on assays, requesting object with ID = 709
Growth of Col-0 in 12hL:12hD (Experimental Assay) -- Cultivation Experiment,Cultivation experiment

Data for Figure 2I-2K in Chew et al. PNAS 2014.
Experimental conditions: 21.3 C; 12:12-h light/dark cycle; light intensity, 110 molm2s1;mean daytime CO2 level, 375 ppm. The error bars show the SEs of five plants
Further detail on the experimental conditions is contained in the public record on the BioDare resource, link to follow

Links:http://jermontology.org/ontology/JERMOntology#Cultivation_experiment
http://jermontology.org/ontology/JERMOntology#Technology_type


Creators: 
Calling API for data on people, requesting object with ID = 850


KeyError: 'assay_class'